core5_05_failure_typology_builder.ipynb

목적:
- 실패를 '느낌'이 아니라 유형(class)으로 고정
- 구조 설명용 taxonomy 생성
- 제출용 레포에서는 일부만 선별 가능

주의:
- 규칙 기반 라벨링
- 정답 아님 / 모델 아님

In [2]:
FAILURE_TAXONOMY = {
    "T1_Boundary_Oscillation": "임계 부근에서 HOLD/MUTATE가 반복적으로 진동",
    "T2_Conflicting_Signals": "pred_score_delta와 누적 맥락이 상충",
    "T3_Burst_Cascade": "연속 MUTATE가 길게 발생하며 개입 폭주",
    "T4_Recovery_Overshoot": "회복 직전(pred 개선 직후)에 개입 발생",
    "T5_Rule_Interaction_Explosion": "규칙 강화 이후 toggle/burst 동시 증가"
}

FAILURE_TAXONOMY

{'T1_Boundary_Oscillation': '임계 부근에서 HOLD/MUTATE가 반복적으로 진동',
 'T2_Conflicting_Signals': 'pred_score_delta와 누적 맥락이 상충',
 'T3_Burst_Cascade': '연속 MUTATE가 길게 발생하며 개입 폭주',
 'T4_Recovery_Overshoot': '회복 직전(pred 개선 직후)에 개입 발생',
 'T5_Rule_Interaction_Explosion': '규칙 강화 이후 toggle/burst 동시 증가'}

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
import json

TRACE_PATH = Path("../artifact/core2/prediction_trace.csv")
COUNTERFACTUAL_PATH = Path("../artifact/core5/counterfactual_results.csv")

trace_df = pd.read_csv(TRACE_PATH)
cf_df = pd.read_csv(COUNTERFACTUAL_PATH)

print("trace:", trace_df.shape)
print("counterfactual:", cf_df.shape)

trace_df.head()

def compute_toggle_events(decisions):
    prev = decisions.shift(1)
    toggle = (decisions != prev)
    toggle.iloc[0] = False
    return toggle


def compute_burst_segments(decisions):
    bursts = []
    current = 0
    for i, d in enumerate(decisions):
        if d == "MUTATE":
            current += 1
        else:
            if current > 0:
                bursts.append(current)
                current = 0
    if current > 0:
        bursts.append(current)
    return bursts # 공통 지표 계산 함수

trace: (60, 8)
counterfactual: (6, 8)


In [4]:
def classify_failure_types(trace_sub):
    decisions = trace_sub["decision"].reset_index(drop=True)
    deltas = trace_sub["pred_score_delta"].reset_index(drop=True)
    
    toggle_flags = compute_toggle_events(decisions)
    toggle_rate = toggle_flags.sum() / len(decisions)
    
    bursts = compute_burst_segments(decisions)
    burst_mean = np.mean(bursts) if bursts else 0
    burst_max = max(bursts) if bursts else 0
    
    labels = []
    example_steps = {}
    
    # T1: Boundary oscillation
    if toggle_rate > 0.25:
        labels.append("T1_Boundary_Oscillation")
        idx = toggle_flags[toggle_flags].index[:3].tolist()
        example_steps["T1_Boundary_Oscillation"] = idx
    
    # T3: Burst cascade
    if burst_max >= 3:
        labels.append("T3_Burst_Cascade")
        example_steps["T3_Burst_Cascade"] = bursts[:3]
    
    # T4: Recovery overshoot
    overshoot_idx = []
    for i in range(1, len(deltas)):
        if deltas.iloc[i] > 0 and decisions.iloc[i] == "MUTATE":
            overshoot_idx.append(i)
    if len(overshoot_idx) > 0:
        labels.append("T4_Recovery_Overshoot")
        example_steps["T4_Recovery_Overshoot"] = overshoot_idx[:3]
    
    # T2: Conflicting signals
    conflict_idx = []
    for i in range(1, len(deltas)):
        if deltas.iloc[i] < 0 and decisions.iloc[i] == "MUTATE":
            conflict_idx.append(i)
    if len(conflict_idx) > 0:
        labels.append("T2_Conflicting_Signals")
        example_steps["T2_Conflicting_Signals"] = conflict_idx[:3]
    
    return {
        "labels": labels,
        "toggle_rate": toggle_rate,
        "burst_max": burst_max,
        "example_steps": example_steps
    } # 실패 유형 판별 규칙

In [5]:
failure_catalog = []

for ab in trace_df["antibody_key"].unique():
    sub = trace_df[trace_df["antibody_key"] == ab].sort_values("step")
    result = classify_failure_types(sub)
    
    failure_catalog.append({
        "antibody_key": ab,
        "failure_types": result["labels"],
        "toggle_rate": result["toggle_rate"],
        "burst_max": result["burst_max"],
        "example_steps": result["example_steps"]
    })

failure_catalog # 체별 실패 유형 분류

[{'antibody_key': 'GDPa1-001',
  'failure_types': ['T1_Boundary_Oscillation',
   'T3_Burst_Cascade',
   'T4_Recovery_Overshoot'],
  'toggle_rate': 0.45,
  'burst_max': 3,
  'example_steps': {'T1_Boundary_Oscillation': [1, 4, 7],
   'T3_Burst_Cascade': [1, 3, 1],
   'T4_Recovery_Overshoot': [4, 5, 6]}},
 {'antibody_key': 'GDPa1-045',
  'failure_types': ['T1_Boundary_Oscillation',
   'T3_Burst_Cascade',
   'T4_Recovery_Overshoot'],
  'toggle_rate': 0.3,
  'burst_max': 6,
  'example_steps': {'T1_Boundary_Oscillation': [2, 4, 5],
   'T3_Burst_Cascade': [2, 1, 6],
   'T4_Recovery_Overshoot': [1, 4, 8]}},
 {'antibody_key': 'GDPa1-183',
  'failure_types': ['T1_Boundary_Oscillation',
   'T3_Burst_Cascade',
   'T4_Recovery_Overshoot'],
  'toggle_rate': 0.35,
  'burst_max': 3,
  'example_steps': {'T1_Boundary_Oscillation': [1, 8, 9],
   'T3_Burst_Cascade': [1, 1, 3],
   'T4_Recovery_Overshoot': [8, 13, 14]}}]

In [6]:
rule_explosion_cases = cf_df[
    (cf_df["toggle_rate"] > 0.3) &
    (cf_df["burst_mean"] > 1.5)
]

t5_cases = rule_explosion_cases["antibody_key"].unique().tolist()
t5_cases

for entry in failure_catalog:
    if entry["antibody_key"] in t5_cases:
        entry["failure_types"].append("T5_Rule_Interaction_Explosion")
        entry["example_steps"]["T5_Rule_Interaction_Explosion"] = ["counterfactual"]

# Counterfactual 결과 기반 T5 추가

In [7]:
failure_catalog_structured = {
    "taxonomy": FAILURE_TAXONOMY,
    "entries": failure_catalog,
    "notes": [
        "라벨은 구조 서술용이며 정답 분류가 아님",
        "단일 실험이 여러 실패 유형을 동시에 가질 수 있음",
        "대표 step은 시각화/슬라이드 캡처 용도"
    ]
}

failure_catalog_structured #실패 카탈로그 구조화

OUTPUT_DIR = Path("../artifact/core5")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

out_path = OUTPUT_DIR / "failure_catalog.json"

with open(out_path, "w") as f:
    json.dump(failure_catalog_structured, f, indent=2)

out_path

PosixPath('../artifact/core5/failure_catalog.json')